In [3]:
# Install YOLOv8
!pip install -q ultralytics

import cv2
import numpy as np
import os
import time
from ultralytics import YOLO
from collections import defaultdict

model = YOLO("yolov8m.pt")

# Prepare input and output paths
input_video = "/content/car.mp4"
output_path = '/content/car_BoT_SORT.mp4'

previous_ids = set()
frame_id = 0

# Start measuring time
start_time = time.time()
num_frames = 0

# Tracking + Writing
out = None  # define writer later dynamically

for result in model.track(
    source=input_video,
    persist=True,
    stream=True,
    show=False,
    tracker="/content/botsort.yaml",
    classes=2
):
    frame = result.orig_img.copy()

    # Create VideoWriter once based on frame size
    if out is None:
        height, width = frame.shape[:2]
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps_input = 30
        out = cv2.VideoWriter(output_path, fourcc, fps_input, (width, height))

    current_ids = set()

    if result.boxes.id is not None:
        ids = result.boxes.id.int().cpu().tolist()
        boxes = result.boxes.xyxy.cpu().numpy()
        confs = result.boxes.conf.cpu().numpy()

        for id, box, conf in zip(ids, boxes, confs):
            current_ids.add(id)
            x1, y1, x2, y2 = box.astype(int)

            # Draw bounding boxes and ID
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f'ID: {id}'
            cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    # Detect NEW and MISSING IDs
    new_ids = current_ids - previous_ids
    missing_ids = previous_ids - current_ids

    for new_id in new_ids:
        print(f"[Frame {frame_id}] New Object Appeared: ID {new_id}")

    for missing_id in missing_ids:
        print(f"[Frame {frame_id}] Object Missing: ID {missing_id}")

    previous_ids = current_ids.copy()
    frame_id += 1
    num_frames += 1

    # Write frame
    out.write(frame)

# Release writer
if out:
    out.release()

# End measuring time
end_time = time.time()
total_time = end_time - start_time
fps_achieved = num_frames / total_time

print(f"✅ Done! Output saved to {output_path}")
print(f"📈 Processed {num_frames} frames in {total_time:.2f} seconds")
print(f"🚀 Achieved FPS: {fps_achieved:.2f}")


video 1/1 (frame 1/1211) /content/car.mp4: 384x640 3 cars, 26.2ms
[Frame 0] New Object Appeared: ID 1
[Frame 0] New Object Appeared: ID 2
[Frame 0] New Object Appeared: ID 3
video 1/1 (frame 2/1211) /content/car.mp4: 384x640 3 cars, 25.3ms
video 1/1 (frame 3/1211) /content/car.mp4: 384x640 3 cars, 24.3ms
video 1/1 (frame 4/1211) /content/car.mp4: 384x640 3 cars, 24.3ms
video 1/1 (frame 5/1211) /content/car.mp4: 384x640 3 cars, 24.3ms
video 1/1 (frame 6/1211) /content/car.mp4: 384x640 3 cars, 24.3ms
video 1/1 (frame 7/1211) /content/car.mp4: 384x640 3 cars, 20.6ms
video 1/1 (frame 8/1211) /content/car.mp4: 384x640 3 cars, 18.9ms
video 1/1 (frame 9/1211) /content/car.mp4: 384x640 3 cars, 18.9ms
video 1/1 (frame 10/1211) /content/car.mp4: 384x640 3 cars, 18.9ms
video 1/1 (frame 11/1211) /content/car.mp4: 384x640 3 cars, 19.0ms
video 1/1 (frame 12/1211) /content/car.mp4: 384x640 3 cars, 18.9ms
video 1/1 (frame 13/1211) /content/car.mp4: 384x640 3 cars, 14.6ms
video 1/1 (frame 14/1211) /co